In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install emoji

In [ ]:
!git clone https://github.com/huggingface/transformers.git

fatal: destination path 'transformers' already exists and is not an empty directory.


In [ ]:
%cd transformers

/content/transformers


In [ ]:
!pip3 install --upgrade .

Processing /content/transformers
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-4.16.0.dev0-py3-none-any.whl size=3377877 sha256=e3ccd95e37966fbf5431a4181312b12466385fcc43f986914c311653b4bdf195
  Stored in directory: /tmp/pip-ephem-wheel-cache-77gig9bi/wheels/49/62/f4/6730819eed4e6468662b1519bf3bf46419b2335990c77f8767
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.16.0.dev0
    Uninstalling trans

In [ ]:
import os
import json
import random
import time
import datetime
import torch
import argparse
import numpy as np
import pandas as pd
from torch.nn import functional as F
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score, confusion_matrix
from transformers import get_linear_schedule_with_warmup,AdamW,AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import TensorDataset,DataLoader, RandomSampler, SequentialSampler, Dataset

In [ ]:
def convert_label(label):
    if label == "desirability":
        return 0
    elif label == "enabling context":
        return 1
    elif label == "can do":
        return 2
    elif label == "buzz":
        return 3
    elif label == "invitation":
        return 4
    else:
        raise Exception("label classes must be 'desirability' or 'enabling context' or 'buzz' or 'can do' or 'invitation'")

In [ ]:
%cd ..

/content


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Research Track/Dataset/Experiment Dataset/Dataset_V4.csv')
df

,Unnamed: 0,decision,created_at,id,id_str,full_text,source,retweet_count,favorite_count,reply,username_length,user_location
0,0,enabling context,2021-01-30 02:40:25+00:00,1355345090022154241,1355345090022154241,@science_barbie @priyology Cyanobacteria (also...,Twitter Web App,0,0,True,8,Unknown
1,1,enabling context,2021-01-30 01:05:16+00:00,1355321147701686273,1355321147701686273,New insights into coral bleaching by internati...,Twitter Web App,1,1,False,10,Unknown
2,2,buzz,2021-01-29 22:49:17+00:00,1355286925809094659,1355286925809094659,"Friday Tally:\n\n2 dives, continued coral blea...",Twitter for iPhone,0,1,False,11,Unknown
3,3,desirability,2021-01-29 22:09:20+00:00,1355276872410681348,1355276872410681348,Coral bleaching is so sad because they just be...,Twitter for iPhone,0,2,False,8,Unknown
4,4,desirability,2021-01-29 21:34:44+00:00,1355268165148815360,1355268165148815360,predicts that at the current rate of emissions...,Twitter Web App,0,0,True,13,Canada
...,...,...,...,...,...,...,...,...,...,...,...,...
1217,1217,buzz,2021-05-17 19:30:12+00:00,1394374712835264513,1394374712835264513,#ClimateChampions developed learning resources...,Twitter Web App,4,14,False,10,United States
1218,1218,desirability,2021-05-17 17:37:33+00:00,1394346363736018949,1394346363736018949,"""For all we hear about #coral bleaching &amp; ...",Twitter Web App,11,24,False,14,Unknown
1219,1219,enabling context,2021-05-17 17:01:39+00:00,1394337327930122250,1394337327930122250,Corals are critical. Track coral bleaching in ...,Hootsuite Inc.,10,29,False,13,United States
1220,1220,enabling context,2021-05-17 16:09:02+00:00,1394324089200779268,1394324089200779268,The number one root cause of coral reefs facin...,Twitter for iPhone,1,0,True,12,South Korea


In [ ]:
df = df[['decision','full_text']]
df = df.rename(columns={'full_text':'text','decision':'label'})

In [ ]:
df['label'] = df['label'].map({
    'desirability': 0, 
    'enabling context': 1,
    'can do': 2,
    'buzz':3,
    'invitation': 4
    })

In [ ]:
df['label'] = df['label'].astype(int)

In [ ]:
df

,label,text
0,1,@science_barbie @priyology Cyanobacteria (also...
1,1,New insights into coral bleaching by internati...
2,3,"Friday Tally:\n\n2 dives, continued coral blea..."
3,0,Coral bleaching is so sad because they just be...
4,0,predicts that at the current rate of emissions...
...,...,...
1217,3,#ClimateChampions developed learning resources...
1218,0,"""For all we hear about #coral bleaching &amp; ..."
1219,1,Corals are critical. Track coral bleaching in ...
1220,1,The number one root cause of coral reefs facin...


In [ ]:
def cleansing(text, stopword = None):
    # remove hastag
    text = ' '.join(word for word in text.split(' ') if not word.startswith('#'))
    # remove mention
    text = ' '.join(word for word in text.split(' ') if not word.startswith('@'))
    # remove URL
    text = ' '.join(word for word in text.split(' ') if not word.startswith('https:'))

    # case folding to lowercase
    text = text.lower()
    # split by space
    word_list = text.split()
    # pick only word with length greater than 2
    word_list = [word for word in word_list if len(word) > 2]
    # pick only alpha-numeric word 
    word_list = [word for word in word_list if word.isalnum()]

    if stopword == None:
        text = ' '.join(word_list)
    else:
        word_list = [word for word in word_list if word not in stopword]
        text = ' '.join(word_list)
                
    return text

In [ ]:
for i in range(len(df)):
    df.loc[i, 'text'] = cleansing(df.loc[i, 'text'])

In [ ]:
df

,label,text
0,1,cyanobacteria called are microorganisms that c...
1,1,new insights into coral bleaching internationa...
2,3,friday continued coral bleaching documentation...
3,0,coral bleaching sad because they just there vi...
4,0,predicts that the current rate there likely in...
...,...,...
1217,3,developed learning resources and curriculum fo...
1218,0,all hear about bleaching other early these imp...
1219,1,corals are track coral bleaching the pacific w...
1220,1,the number one root cause coral reefs facing d...


In [ ]:
def bert_encode(df, tokenizer):
    input_ids = []
    attention_masks = []
    for sentence in df[["text"]].values:
        sentence = sentence.item()
        encoded_dict = tokenizer.encode_plus(
                            sentence,                      
                            add_special_tokens = True,  
                            max_length = 72,
                            pad_to_max_length = True,
                            truncation = True,
                            return_attention_mask = True,   
                            return_tensors = 'pt',    
                    )
           
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    inputs = {
    'input_word_ids': input_ids,
    'input_mask': attention_masks}

    return inputs

In [ ]:
def test_encode(sentence):
    tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False, normalization=True)

    encoded_dict = tokenizer.encode_plus(
                        sentence,                      
                        add_special_tokens = True,
                        max_length = 72,
                        pad_to_max_length = True,
                        truncation = True,
                        return_attention_mask = True,   
                        return_tensors = 'pt',    
                )
           
    return encoded_dict['input_ids']

In [ ]:
def test_decode(tokens):
    tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False, normalization=True)
    return tokenizer.convert_ids_to_tokens(tokens)

In [ ]:
text_encoded =[]
for i in range(0,len(df)):
  te = test_encode(df['text'][i])
  text_encoded.append(te)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2225: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
S

In [ ]:
text_encoded

[tensor([[    0,  6936,   725,  4363,  1396, 40782,   488,    41,  8368, 45723,
             25,    56,  6052,   953,  1708,   638,  6355,   109,   248,  6052,
             64,  1549, 59266,     6,   311,  1007,    13,  1780, 47476, 42398,
          36507,     2,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1]]),
 tensor([[    0,   127, 17562,   187, 36507, 24998,  4544,  3227, 12027, 28522,
          47903, 15825, 36464,  1330, 17744,  2091,  1565,   980, 32971,  2579,
           2624,  1215, 38459, 13940, 34517, 60741,  2161, 14337,   182,   213,
          11491,     6,     2,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             

In [ ]:
t = []
for i in range(0,len(df)):
  te = np.array(text_encoded[i][0])
  t.append(te)

In [ ]:
t

[array([    0,  6936,   725,  4363,  1396, 40782,   488,    41,  8368,
        45723,    25,    56,  6052,   953,  1708,   638,  6355,   109,
          248,  6052,    64,  1549, 59266,     6,   311,  1007,    13,
         1780, 47476, 42398, 36507,     2,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1]),
 array([    0,   127, 17562,   187, 36507, 24998,  4544,  3227, 12027,
        28522, 47903, 15825, 36464,  1330, 17744,  2091,  1565,   980,
        32971,  2579,  2624,  1215, 38459, 13940, 34517, 60741,  2161,
        14337,   182,   213, 11491,     6,     2,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
    

In [ ]:
from pandas import DataFrame

new_df = DataFrame(t)
new_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71
0,0,6936,725,4363,1396,40782,488,41,8368,45723,25,56,6052,953,1708,638,6355,109,248,6052,64,1549,59266,6,311,1007,13,1780,47476,42398,36507,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,0,127,17562,187,36507,24998,4544,3227,12027,28522,47903,15825,36464,1330,17744,2091,1565,980,32971,2579,2624,1215,38459,13940,34517,60741,2161,14337,182,213,11491,6,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,0,3426,4526,36507,24998,4544,36872,5949,78,217,89,37471,1070,94,5757,177,166,16350,1006,1155,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,0,36507,24998,4544,573,153,59,45,99,2358,8509,252,66,1323,133,668,1006,3998,2389,130,5601,6,283,784,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,0,37480,25,6,1868,2425,99,2056,3749,6,271,3885,7014,4496,5725,10346,237,3790,629,1117,16318,13,1396,11982,11562,48,248,1340,2237,22376,14762,16025,138,16025,1307,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1217,0,4716,2265,5961,13,23200,19,5469,1690,1178,5725,13,32462,6,13579,3885,41,20474,2031,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1218,0,48,473,62,24998,4544,185,582,198,24494,41,121,45,1875,20720,367,138,70,590,146,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1219,0,3093,2467,41,1512,36507,24998,4544,6,52217,30,6,36507,8517,6541,2707,24998,4544,560,50,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1220,0,6,644,63,7223,311,36507,8517,6541,6484,27993,4682,6,3384,45965,2285,22345,6,422,5711,30137,6,42780,17446,6,36507,24998,4544,36,3924,923,6,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [ ]:
df_final = pd.concat([df,new_df],axis=1)

In [ ]:
from nltk.cluster import KMeansClusterer
import nltk

def clustering_question(data,NUM_CLUSTERS = 5):

    sentences = df_final['text']

    X = t

    kclusterer = KMeansClusterer(
        NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance,
        repeats=25,avoid_empty_clusters=True)

    assigned_clusters = kclusterer.cluster(X, assign_clusters=True)

    df_final['cluster'] = pd.Series(assigned_clusters, index=data.index)
    df_final['centroid'] = data['cluster'].apply(lambda x: kclusterer.means()[x])

    return data, assigned_clusters

In [ ]:
clustering_question(df_final)

(      label  ...                                           centroid
 0         1  ...  [0.0, 4093.435044693966, 6347.413898919584, 42...
 1         1  ...  [0.0, 4086.570000000174, 4579.360000141459, 62...
 2         3  ...  [0.0, 4135.303225817123, 13316.825806146348, 3...
 3         0  ...  [0.0, 20919.85926318018, 14609.562959584948, 4...
 4         0  ...  [0.0, 20919.85926318018, 14609.562959584948, 4...
 ...     ...  ...                                                ...
 1217      3  ...  [0.0, 2774.842105695009, 4334.184211388586, 33...
 1218      0  ...  [0.0, 4086.570000000174, 4579.360000141459, 62...
 1219      1  ...  [0.0, 4086.570000000174, 4579.360000141459, 62...
 1220      1  ...  [0.0, 4093.435044693966, 6347.413898919584, 42...
 1221      1  ...  [0.0, 4086.570000000174, 4579.360000141459, 62...
 
 [1222 rows x 76 columns],
 [4,
  2,
  1,
  0,
  0,
  3,
  1,
  4,
  0,
  4,
  1,
  3,
  0,
  2,
  0,
  3,
  0,
  0,
  4,
  4,
  4,
  0,
  3,
  0,
  2,
  3,
  3,
  0,
  4

In [ ]:
df_final

,label,text,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,cluster,centroid
0,1,cyanobacteria called are microorganisms that c...,0,6936,725,4363,1396,40782,488,41,8368,45723,25,56,6052,953,1708,638,6355,109,248,6052,64,1549,59266,6,311,1007,13,1780,47476,42398,36507,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,4,"[0.0, 4093.435044693966, 6347.413898919584, 42..."
1,1,new insights into coral bleaching internationa...,0,127,17562,187,36507,24998,4544,3227,12027,28522,47903,15825,36464,1330,17744,2091,1565,980,32971,2579,2624,1215,38459,13940,34517,60741,2161,14337,182,213,11491,6,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,"[0.0, 4086.570000000174, 4579.360000141459, 62..."
2,3,friday continued coral bleaching documentation...,0,3426,4526,36507,24998,4544,36872,5949,78,217,89,37471,1070,94,5757,177,166,16350,1006,1155,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,"[0.0, 4135.303225817123, 13316.825806146348, 3..."
3,0,coral bleaching sad because they just there vi...,0,36507,24998,4544,573,153,59,45,99,2358,8509,252,66,1323,133,668,1006,3998,2389,130,5601,6,283,784,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,"[0.0, 20919.85926318018, 14609.562959584948, 4..."
4,0,predicts that the current rate there likely in...,0,37480,25,6,1868,2425,99,2056,3749,6,271,3885,7014,4496,5725,10346,237,3790,629,1117,16318,13,1396,11982,11562,48,248,1340,2237,22376,14762,16025,138,16025,1307,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,"[0.0, 20919.85926318018, 14609.562959584948, 4..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1217,3,developed learning resources and curriculum fo...,0,4716,2265,5961,13,23200,19,5469,1690,1178,5725,13,32462,6,13579,3885,41,20474,2031,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3,"[0.0, 2774.842105695009, 4334.184211388586, 33..."
1218,0,all hear about bleaching other early these imp...,0,48,473,62,24998,4544,185,582,198,24494,41,121,45,1875,20720,367,138,70,590,146,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,"[0.0, 4086.570000000174, 4579.360000141459, 62..."
1219,1,corals are track coral bleaching the pacific w...,0,3093,2467,41,1512,36507,24998,4544,6,52217,30,6,36507,8517,6541,2707,24998,4544,560,50,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,"[0.0, 4086.570000000174, 4579.360000141459, 62..."
1220,1,the number one root cause coral reefs facing d...,0,6,644,63,7223,311,36507,8517,6541,6484,27993,4682,6,3384,45965,2285,22345,6,422,5711,30137,6,42780,17446,6,36507,24998,4544,36,3924,923,6,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,4,"[0.0, 4093.435044693966, 6347.413898919584, 42..."


In [ ]:
cluster_zero = df_final.loc[df_final['cluster'] == 0]
cluster_zero

,label,text,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,cluster,centroid
3,0,coral bleaching sad because they just there vi...,0,36507,24998,4544,573,153,59,45,99,2358,8509,252,66,1323,133,668,1006,3998,2389,130,5601,6,283,784,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,"[0.0, 20919.85926318018, 14609.562959584948, 4..."
4,0,predicts that the current rate there likely in...,0,37480,25,6,1868,2425,99,2056,3749,6,271,3885,7014,4496,5725,10346,237,3790,629,1117,16318,13,1396,11982,11562,48,248,1340,2237,22376,14762,16025,138,16025,1307,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,"[0.0, 20919.85926318018, 14609.562959584948, 4..."
8,4,january 30th 2021 guest lecture franziska ston...,0,31056,9842,26568,4830,9152,42822,685,924,1185,1061,8264,36507,16399,2116,5110,13,36507,24998,4544,7157,25090,934,19,6225,10411,9842,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,"[0.0, 20919.85926318018, 14609.562959584948, 4..."
12,1,american lobster postlarvae alter gene regulat...,0,8662,23667,7249,5477,56727,17895,15798,18869,2166,5725,10346,13,1396,11982,11562,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,"[0.0, 20919.85926318018, 14609.562959584948, 4..."
14,3,owe our children separate atmospheric climate ...,0,6504,123,994,5628,53246,5211,3885,10346,5725,10346,13,7595,3330,1084,3384,13,53,27072,4982,13,6151,7574,25,239,41,252,112,132,572,198,2990,10437,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,"[0.0, 20919.85926318018, 14609.562959584948, 4..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1186,3,scientists launch tool detect coral,0,10544,3860,5376,27525,36507,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,"[0.0, 20919.85926318018, 14609.562959584948, 4..."
1189,1,coral evidences coral bleaching powerful way t...,0,36507,37490,9461,36507,24998,4544,3409,154,307,86,9504,42,101,179,4309,360,307,147,117,95,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,"[0.0, 20919.85926318018, 14609.562959584948, 4..."
1203,0,xplosive landfill coral foodchain recycling th...,0,577,18274,7626,5932,3524,36507,24159,5685,23125,33,2438,5211,42,112,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,"[0.0, 20919.85926318018, 14609.562959584948, 4..."
1210,0,coral holds the when releases that carbon back...,0,36507,4714,6,64,7651,25,12715,107,187,6,36507,24998,4544,4288,6,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,"[0.0, 20919.85926318018, 14609.562959584948, 4..."


In [ ]:
cluster_zero['label'].value_counts()
# cluster 0 = Cluster Enabling Context

1    111
0     83
3     61
4      9
2      6
Name: label, dtype: int64

In [ ]:
cluster_one = df_final.loc[df_final['cluster'] == 1]
cluster_one

,label,text,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,cluster,centroid
2,3,friday continued coral bleaching documentation...,0,3426,4526,36507,24998,4544,36872,5949,78,217,89,37471,1070,94,5757,177,166,16350,1006,1155,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,"[0.0, 4135.303225817123, 13316.825806146348, 3..."
6,1,happening coral bleaching,0,1774,36507,24998,4544,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,"[0.0, 4135.303225817123, 13316.825806146348, 3..."
10,1,research reveals genetic response ocean warmin...,0,2150,7329,19892,2166,5725,10346,13,1396,11982,11562,8662,23667,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,"[0.0, 4135.303225817123, 13316.825806146348, 3..."
29,1,whether mass coral bleaching rapidly intensify...,0,2234,4547,36507,24998,4544,15072,33240,13907,2905,8464,2082,25,31849,2714,35468,219,1523,4856,74,6,51733,25,7555,19468,137,5211,431,45,9484,18024,1544,6,121,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,"[0.0, 4135.303225817123, 13316.825806146348, 3..."
31,1,lost its coral reefs 1998 not overfishing but ...,0,590,139,36507,8517,6541,6683,46,1824,8043,42,36507,248,1780,47476,607,1269,10346,11929,13,5211,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,"[0.0, 4135.303225817123, 13316.825806146348, 3..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1183,0,oil coral shark finning and most certainly com...,0,2797,36507,11361,4234,4529,13,226,4289,3396,8043,48,12808,1690,9935,2145,248,151,6542,6,7323,13,4285,151,6151,13,139,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,"[0.0, 4135.303225817123, 13316.825806146348, 3..."
1192,3,global coral restoration program mission creat...,0,3885,36507,24630,2179,4056,2002,13,991,6479,13,20439,42803,8533,15320,36507,8517,6541,284,6,220,259,50,89,265,130,7358,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,"[0.0, 4135.303225817123, 13316.825806146348, 3..."
1202,3,the radical coral rescue plan that paid resear...,0,6,12958,36507,8098,1028,25,1328,34603,2465,36507,24630,19,17981,558,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,"[0.0, 4135.303225817123, 13316.825806146348, 3..."
1208,1,sensitivity the atlantic meridional overturnin...,0,32424,6,55009,3126,11982,20425,1824,2799,31092,13,5211,12598,10804,5725,10346,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,"[0.0, 4135.303225817123, 13316.825806146348, 3..."


In [ ]:
cluster_one['label'].value_counts()
#Cluster 1 = Cluster Enabling Context

1    62
0    44
3    38
4     6
2     5
Name: label, dtype: int64

In [ ]:
cluster_two = df_final.loc[df_final['cluster'] == 2]
cluster_two

,label,text,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,cluster,centroid
1,1,new insights into coral bleaching internationa...,0,127,17562,187,36507,24998,4544,3227,12027,28522,47903,15825,36464,1330,17744,2091,1565,980,32971,2579,2624,1215,38459,13940,34517,60741,2161,14337,182,213,11491,6,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,"[0.0, 4086.570000000174, 4579.360000141459, 62..."
13,0,even minecraft reefs safe from the threat cora...,0,132,20197,8517,6541,1309,53,6,4682,36507,24998,4544,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,"[0.0, 4086.570000000174, 4579.360000141459, 62..."
24,0,southern ocean waters are warming faster than ...,0,6321,5725,11929,41,10346,4016,149,11775,2330,41184,1110,156,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,"[0.0, 4086.570000000174, 4579.360000141459, 62..."
38,1,major discovery helps explain coral via proces...,0,1690,13668,2820,1966,36507,156,2156,17605,18262,5725,18402,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,"[0.0, 4086.570000000174, 4579.360000141459, 62..."
40,3,coral restoration nfl green and partners resto...,0,36507,24630,26906,1934,13,8284,41641,961,36507,52194,2137,1485,985,9832,13018,13,10544,252,6,25177,28255,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,"[0.0, 4086.570000000174, 4579.360000141459, 62..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1205,1,rangers researcher qualified padi open water s...,0,24861,34603,9925,581,11473,634,791,2228,25696,57305,868,11217,6,2228,25696,868,70,16937,6,24861,6488,36507,17627,47514,6,2022,25,70,2953,24082,33,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,"[0.0, 4086.570000000174, 4579.360000141459, 62..."
1207,3,aims restore the equivalent 148 olympic swimmi...,0,15513,15320,6,10313,26469,41527,5794,25181,36507,52194,1068,675,6,3860,6,4164,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,"[0.0, 4086.570000000174, 4579.360000141459, 62..."
1218,0,all hear about bleaching other early these imp...,0,48,473,62,24998,4544,185,582,198,24494,41,121,45,1875,20720,367,138,70,590,146,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,"[0.0, 4086.570000000174, 4579.360000141459, 62..."
1219,1,corals are track coral bleaching the pacific w...,0,3093,2467,41,1512,36507,24998,4544,6,52217,30,6,36507,8517,6541,2707,24998,4544,560,50,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,"[0.0, 4086.570000000174, 4579.360000141459, 62..."


In [ ]:
cluster_two['label'].value_counts()
#Cluster 2 = Cluster Enabling Context

1    73
3    52
0    51
2    13
4    11
Name: label, dtype: int64

In [ ]:
cluster_three = df_final.loc[df_final['cluster'] == 3]
cluster_three

,label,text,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,cluster,centroid
5,0,warming the ocean affects marine species cause...,0,10346,6,5725,14762,16025,4110,311,36507,24998,4544,6,2116,22513,10406,19,6,10346,28176,237,6980,89,6953,1155,21954,15501,528,6,4856,4344,90,3694,25,6,28176,36,29588,89,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3,"[0.0, 2774.842105695009, 4334.184211388586, 33..."
11,3,going special dive today with the coral restor...,0,117,985,11382,128,30,6,36507,24630,7970,914,84,12586,52194,10306,9219,28255,3093,2467,53,130,28812,50,6,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3,"[0.0, 2774.842105695009, 4334.184211388586, 33..."
15,1,need clear when sediment pollutants enter the ...,0,114,1673,64,1169,32074,33413,62175,2928,6,59,3626,23869,638,38807,791,1956,3626,42,41641,8100,56,371,89,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3,"[0.0, 2774.842105695009, 4334.184211388586, 33..."
22,2,get know more about the latest coral restorati...,0,51,68,89,62,6,1405,36507,24630,7970,1133,123,1133,4798,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3,"[0.0, 2774.842105695009, 4334.184211388586, 33..."
25,3,the featured your work surveying coral bleachi...,0,6,4738,44,157,53446,776,36507,24998,4544,1417,6,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3,"[0.0, 2774.842105695009, 4334.184211388586, 33..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1194,3,thanks technology and lot hard the first coral...,0,432,3670,13,318,301,6,160,36507,17627,1135,77,33,127,5376,5490,1463,913,5785,2682,13,15320,36507,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3,"[0.0, 2774.842105695009, 4334.184211388586, 33..."
1206,1,where the ocean warming this plot compares the...,0,209,6,5725,10346,33,6094,25456,6,127,3330,7809,7014,9559,3809,6578,30,6,289,6,10346,4642,1153,6,3420,5300,90,108,63,6,4164,6,4476,18262,4476,10347,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3,"[0.0, 2774.842105695009, 4334.184211388586, 33..."
1209,2,you target red populations but coalesce studen...,0,14,4046,1007,25900,42,1356,638,783,4157,1343,25,5707,36507,183,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3,"[0.0, 2774.842105695009, 4334.184211388586, 33..."
1211,1,issue from open for warming and acidification ...,0,1402,53,634,19,10346,13,1396,11982,11562,11409,2737,614,806,34277,17938,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3,"[0.0, 2774.842105695009, 4334.184211388586, 33..."


In [ ]:
cluster_three['label'].value_counts()
#Cluster 3 = Cluster Desirability

0    81
3    74
1    73
2    21
4    17
Name: label, dtype: int64

In [ ]:
cluster_four = df_final.loc[df_final['cluster'] == 4]
cluster_four

,label,text,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,cluster,centroid
0,1,cyanobacteria called are microorganisms that c...,0,6936,725,4363,1396,40782,488,41,8368,45723,25,56,6052,953,1708,638,6355,109,248,6052,64,1549,59266,6,311,1007,13,1780,47476,42398,36507,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,4,"[0.0, 4093.435044693966, 6347.413898919584, 42..."
7,4,going the biggest coral restoration event happ...,0,117,6,1345,36507,24630,1278,1774,220,93,17488,1075,33,37090,1544,1133,123,36507,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,4,"[0.0, 4093.435044693966, 6347.413898919584, 42..."
9,1,ban that contains and octinoxate went into eff...,0,3306,25,8390,13,2702,543,7054,577,760,436,187,3599,37082,5923,3363,13,2702,543,7054,577,760,41,11040,20674,25,56,311,4237,15041,30,37698,20674,32681,13,7043,37554,41,16651,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,4,"[0.0, 4093.435044693966, 6347.413898919584, 42..."
18,1,wonderful see cover current sea anemone tentac...,0,1576,95,1355,1868,3330,725,1254,63,20323,23479,270,6,1015,10611,37239,50177,1961,48,6,8368,7801,60935,776,6,725,1254,63,2820,559,36507,24998,4544,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,4,"[0.0, 4093.435044693966, 6347.413898919584, 42..."
19,1,ocean warming may exacerbate their effects spe...,0,5725,10346,386,1780,47476,42398,130,6021,4110,30,2380,5664,12647,30,5672,791,7014,13,31645,33109,1857,53853,35307,12697,258,33,2290,8554,3749,6066,3273,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,4,"[0.0, 4093.435044693966, 6347.413898919584, 42..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1204,0,weather events induced climate and other envir...,0,1155,2355,29286,5211,13,185,13217,431,2538,21954,12133,23100,638,6355,13,11040,13,8368,45471,198,9935,41,7982,21257,5725,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,4,"[0.0, 4093.435044693966, 6347.413898919584, 42..."
1213,0,co2 sequestering the ocean creates compounds t...,0,1356,119,1169,23105,4825,6,5725,10834,43929,25,1118,6097,187,35552,9570,6,239,59231,4450,13,14602,18040,4110,56,46,294,198,5430,36507,36507,13,22979,84,6638,1356,119,94,19,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,4,"[0.0, 4093.435044693966, 6347.413898919584, 42..."
1215,1,species able swap their algae for more strains...,0,4110,503,14230,130,638,38807,19,89,50711,41,89,2056,4702,350,52315,41765,42,33,38,121,6,861,27745,248,2376,2386,41,2133,1101,13,5725,10346,18768,2703,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,4,"[0.0, 4093.435044693966, 6347.413898919584, 42..."
1216,3,these are really exciting results previous stu...,0,198,41,116,3457,2213,4006,6320,36,408,5490,26334,13850,1505,1799,425,3330,1556,6384,48270,19054,1893,13,48598,42,6,32490,123,36507,24630,32537,4577,15772,5437,19,6,896,52194,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,4,"[0.0, 4093.435044693966, 6347.413898919584, 42..."


In [ ]:
cluster_four['label'].value_counts()
#Cluster 4 = Cluster Desirability

0    114
1    110
3     83
4     16
2      8
Name: label, dtype: int64